# HDS5210-2020 Midterm

In the midterm, you're going to focus on using the programming skills that you've developed so far to build a calculator for the Apache II scoring system for ICU Mortality.  
* https://www.mdcalc.com/apache-ii-score#evidence
* https://reference.medscape.com/calculator/apache-ii-scoring-system

For the midterm, we'll be building a calculator for the Apache II score and then running that against a patient file that's available to you out on the internet.  This will be broken down into three main steps:
1. Create your JSON file to encapsulate all of the calculation rules for Apache II
2. Create functions to calculate the Apache II score using your JSON configuration
3. Create a function to loop over the patients in a file on the internet and calculate Apach II scores for all of them



---

## Part 1: Creating a JSON Rules File

Look at the rules for the Apache II scoring system on the pages above.  The first step in the midterm is to use those rules and create a JSON configuration file as described in the 2019 midterm video.  I've provided a starter file named `apache.json` to get you started.

Inside that file, you'll find placeholders for all of the measures that go into the Apache II scoring model:
* Organ Failure History
* Age
* Temperature
* [pH](https://en.wikipedia.org/wiki/PH)
* Heart rate
* Respiratory rate
* [Sodium](https://www.mayoclinic.org/diseases-conditions/hyponatremia/symptoms-causes/syc-20373711)
* [Potassium](https://www.emedicinehealth.com/hyperkalemia/article_em.htm)
* [Creatinine](https://www.medicalnewstoday.com/articles/322380)
* [Hematocrit](https://labtestsonline.org/tests/hematocrit)
* White Blood Count
* [FiO2](https://www.ausmed.com/cpd/articles/oxygen-flow-rate-and-fio2)
* [PaO2](https://www.verywellhealth.com/partial-pressure-of-oyxgen-pa02-914920)
* [A-a gradient](https://www.ncbi.nlm.nih.gov/books/NBK545153/)


You may need to create a sort of nested set of rules in some cases.  For instance, the rule for Creatinine says to use certain ranges and points in the case of Acute Renal Failure and a different set of points for Chronic Renal Failure.

Similarly, the rule for FiO2 says to use PaO2 to calculate scores if the FiO2 is <50, and to use A-a Gradient if the PaO2 is >50.

When you've created your `apache.json` file, make sure it's in the same directory as this notebook.

### Testing your JSON

The assert() functions below should all run just fine.  If you want to change the names of any of the keys in the JSON I provided you, you may, but you'll also need to update this test code so that it doesn't fail.  Remember, your notebook should be able to run end-to-end before you submit it.

In [1]:
import json

with open('apache.json') as f:
    rules = json.load(f)

In [2]:
rules

{'Organ Failure History': {'Nonoperative': 5,
  'Emergency': 5,
  'Elective': 2,
  'None': 0},
 'Age': [{'min': 0, 'max': 45, 'points': 0},
  {'min': 45, 'max': 55, 'points': 2},
  {'min': 55, 'max': 65, 'points': 3},
  {'min': 65, 'max': 75, 'points': 5},
  {'min': 75, 'max': 999, 'points': 6}],
 'Temperature': [{'min': 41, 'max': 999, 'points': 4},
  {'min': 39, 'max': 41, 'points': 3},
  {'min': 38.5, 'max': 39, 'points': 1},
  {'min': 36, 'max': 38.5, 'points': 0},
  {'min': 34, 'max': 36, 'points': 1},
  {'min': 32, 'max': 34, 'points': 2},
  {'min': 30, 'max': 32, 'points': 3},
  {'min': 0, 'max': 30, 'points': 4}],
 'Heart Rate': [{'min': 180, 'max': 999, 'points': 4},
  {'min': 140, 'max': 180, 'points': 3},
  {'min': 110, 'max': 140, 'points': 2},
  {'min': 70, 'max': 110, 'points': 0},
  {'min': 55, 'max': 70, 'points': 2},
  {'min': 40, 'max': 55, 'points': 3},
  {'min': 0, 'max': 40, 'points': 4}],
 'Respiratory Rate': [{'min': 50, 'max': 999, 'points': 4},
  {'min': 35, 'm

In [3]:
import json

with open('apache.json') as f:
    rules = json.load(f)

assert('Organ Failure History' in rules.keys())
assert('Age' in rules.keys())
assert('Temperature' in rules.keys())
assert('pH' in rules.keys())
assert('Heart Rate' in rules.keys())
assert('Respiratory Rate' in rules.keys())
assert('Sodium' in rules.keys())
assert('Potassium' in rules.keys())
assert('Creatinine' in rules.keys())
assert('Hematocrit' in rules.keys())
assert('White Blood Count' in rules.keys())
assert('FiO2' in rules.keys())

---

## Part 2: Functions to evaluate rules

Write a series of functions, enough to satisfy all of the main criteria that we're using to calculate the Apache II score.  That list is the same as the assert statements above.

* Each of your functions should be well documented.
* Each function should have "config_file" as one of it's parameters.
* Each function should return a numerical score value.
* Similar to what we discussed in the review, if you can generalize some rules, do so.  You should **NOT** end up with one function for each input variable.  If you did that, you'd have a lot of repetative code.

The Glasgow Coma Scale is simply a 1-to-1 score translation.  Simply add the Glasgow Coma Scale value.  So, you don't need to write a function for this. [Glasgow Coma Scale](https://www.cdc.gov/masstrauma/resources/gcs.pdf)

**CORRECTION ADDED 2/29** - The Glasgow Coma Scale points should be calculated as `1 - Glasgow Coma Scale` rather than what I just stated above.  My preference would be that you do the calculation correctly, as per MDCalc, and then use the **corrected** scores files to compare against as noted in Part 4.

## #2A - Organ Failure History

In [4]:
import json

def Organ_Failure_History_score(organ_failure_list,config_file):
    score=0
    config = json.load(open(config_file))
    organ_failure_scores= config.get("Organ Failure History")
    for organ_failure in organ_failure_list:
        if organ_failure in organ_failure_scores:
            score += organ_failure_scores.get(organ_failure)
    
    return score

In [5]:
Organ_Failure_History_CONFIG_FILE = "apache.json"

In [387]:
assert(Organ_Failure_History_score( ["Elective"],Organ_Failure_History_CONFIG_FILE)==2)

# 2B - Age

In [7]:
import json

def Age_score(Age,config_file):
    
    score=0
    config = json.load(open(config_file))
    Age_rules= config.get("Age")
    for rule in Age_rules:
        if float(Age) >= rule.get('min') and float(Age) < rule.get('max'):
            score = rule.get('points')
    
    return score

In [8]:
Age_CONFIG_FILE = "apache.json"

In [389]:
assert(Age_score(6,  Age_CONFIG_FILE)==0)

# 2C - Temperature

In [10]:
import json

def Temperature_score(Temperature,config_file):
    score=0
    config = json.load(open(config_file))
    Temperature_rules= config.get("Temperature")
    for rule in Temperature_rules:
        if float(Temperature) >= rule.get('min') and float(Temperature) < rule.get('max'):
            score = rule.get('points')
    
    return score

In [11]:
Temperature_CONFIG_FILE = "apache.json"

In [390]:
assert(Temperature_score(27,Temperature_CONFIG_FILE)==4)

# 2D - Heart Rate

In [13]:
import json

def Heart_Rate_score(Heart_Rate,config_file):
    score=0
    config = json.load(open(config_file))
    Heart_Rate_rules= config.get("Heart Rate")
    for rule in Heart_Rate_rules:
        if float(Heart_Rate) >= rule.get('min') and float(Heart_Rate) < rule.get('max'):
            score = rule.get('points')
    
    return score

In [14]:
Heart_Rate_CONFIG_FILE = "apache.json"

In [391]:
assert(Heart_Rate_score(174,Heart_Rate_CONFIG_FILE)==3)

# 2E - Respiratory Rate

In [16]:
import json

def Respiratory_Rate_score(Respiratory_Rate,config_file):
    score=0
    config = json.load(open(config_file))
    Respiratory_Rate_rules= config.get("Respiratory Rate")
    for rule in Respiratory_Rate_rules:
        if float(Respiratory_Rate) >= rule.get('min') and float(Respiratory_Rate) < rule.get('max'):
            score = rule.get('points')
    
    return score

In [17]:
Respiratory_Rate_CONFIG_FILE = "apache.json"

In [392]:
assert(Respiratory_Rate_score(10,Respiratory_Rate_CONFIG_FILE)==1)

# 2F - Oxygenation

In [19]:
import json

def Oxygenation_score(FiO2,PaO2,Agradient,config_file):
    score = 0
    config = json.load(open(config_file))
    rules = config.get("FiO2")
    if  float(FiO2) >= rules[0].get('min') and float(FiO2) <rules[0].get('max'):
            PaO2_rules = rules[0].get('PaO2')
            for PaO2_rule in PaO2_rules:
                if float(PaO2) >=PaO2_rule.get('min') and float(PaO2) < PaO2_rule.get('max'):
                    score = PaO2_rule.get('points')
       
    elif  float(FiO2) >= rules[1].get('min') and  float(FiO2) <rules[1].get('max'):
            Agradient_rules = rules[1].get('A-a gradient')
            for Agradient_rule in Agradient_rules:
                if float(Agradient) >=Agradient_rule.get('min') and float(Agradient) < Agradient_rule.get('max'):
                    score = Agradient_rule.get('points')
    
   
    

    return score

In [20]:
PaO2_CONFIG_FILE = "apache.json"

In [393]:
assert(Oxygenation_score(61,59,458,PaO2_CONFIG_FILE)==3)

# 2G - pH

In [22]:
import json

def pH_score(pH,config_file):
    score=0
    config = json.load(open(config_file))
    pH_rules= config.get("pH")
    for rule in pH_rules:
        if float(pH) >= rule.get('min') and float(pH) < rule.get('max'):
            score = rule.get('points')
    
    return score

In [23]:
pH_CONFIG_FILE = "apache.json"

In [394]:
assert(pH_score(7.3,pH_CONFIG_FILE)==2)

# 2H - Sodium

In [25]:
import json

def Sodium_score(Sodium,config_file):
    score=0
    config = json.load(open(config_file))
    Sodium_rules= config.get("Sodium")
    for rule in Sodium_rules:
        if float(Sodium) >= rule.get('min') and float(Sodium) < rule.get('max'):
            score = rule.get('points')
    
    return score

In [26]:
Sodium_CONFIG_FILE = "apache.json"

In [395]:
assert(Sodium_score(170,Sodium_CONFIG_FILE)==3)

# 2I- Potassium

In [28]:
import json

def Potassium_score(Potassium,config_file):
    score=0
    config = json.load(open(config_file))
    Potassium_rules= config.get("Potassium")
    for rule in Potassium_rules:
        if float(Potassium) >= rule.get('min') and float(Potassium) < rule.get('max'):
            score = rule.get('points')
    
    return score

In [29]:
Potassium_CONFIG_FILE = "apache.json"

In [396]:
assert(Potassium_score(7.8,Potassium_CONFIG_FILE)==4)

# 2J -Creatinine

In [31]:
import json

def Creatinine_score(Creatinine,Renal_Failure,config_file):
    score=0
    config = json.load(open(config_file))
    if Renal_Failure=="Acute Renal Failure":
        Creatinine_rules= config.get("Creatinine").get("Acute Renal Failure")
    elif Renal_Failure=="Chronic Renal Failure":
        Creatinine_rules= config.get("Creatinine").get("Chronic Renal Failure")
    for rule in Creatinine_rules:
        if float(Creatinine)>= rule.get('min') and float(Creatinine) < rule.get('max'):
            score = rule.get('points')
    
    return score

In [32]:
Creatinine_CONFIG_FILE = "apache.json"

In [397]:
assert(Creatinine_score(1.6,"Chronic Renal Failure",Creatinine_CONFIG_FILE)==2)

# 2K -Hematocrit

In [34]:
import json

def Hematocrit_score(Hematocrit,config_file):
    score=0
    config = json.load(open(config_file))
    Hematocrit_rules= config.get("Hematocrit")
    for rule in Hematocrit_rules:
        if float(Hematocrit) >= rule.get('min') and float(Hematocrit) < rule.get('max'):
            score = rule.get('points')
    
    return score

In [35]:
Hematocrit_CONFIG_FILE = "apache.json"

In [398]:
assert(Hematocrit_score(3,Hematocrit_CONFIG_FILE)==4)

# 2L -White Blood Count

In [37]:
import json

def White_Blood_score(White_Blood,config_file):
    score=0
    config = json.load(open(config_file))
    White_Blood_rules= config.get("White Blood Count")
    for rule in White_Blood_rules:
        if float(White_Blood) >= rule.get('min') and float(White_Blood) < rule.get('max'):
            score = rule.get('points')
    
    return score

In [38]:
White_Blood_CONFIG_FILE = "apache.json"

In [399]:
assert(White_Blood_score(28,White_Blood_CONFIG_FILE)==2)

### Testing you Functions

Write enough test cases to verify that your functions work for evaulating all of the scoring inputs.  Have at least 3 test cases for each input.

These tests can be written the same as the assertions we've use in previous assignments.  For example, if you a function for `temperature_score` then you write a test case like:

```
assert( temperature_score(37) == 0 )
```

---

## Part 3: Put it all together

Create a new function called `apache_score()` that takes all of the necessary inputs and returns the final Apache II score.  Use any variable names that you want.  For clarity and organization, my recommendation is to create them in the same order as they're documented in the website.

1. Organ Failure History
2. Age
3. Temperature
4. pH 
5. Heart rate
6. Respiratory rate
7. Sodium
8. Potassium
9. Creatinine
10. Acute renal failure
11. Hematocrit
12. White Blood Count
13. Glasgow Coma Scale
14. FiO2
15. PaO2
16. A-a gradient


In [93]:
def apache_score(organ_failure_list,Age,Temperature,Heart_Rate,Respiratory_Rate,FiO2,PaO2,
                Agradient,pH,Sodium,Potassium,Creatinine,Renal_Failure,Hematocrit,White_Blood,Glasgow_Coma_Scale):
    score = 0
    
    config_file = "apache.json"
    score += Organ_Failure_History_score(organ_failure_list,config_file)
    score += Age_score(Age,config_file)
    score += Temperature_score(Temperature,config_file)
    score += Heart_Rate_score(Heart_Rate,config_file)
    score += Respiratory_Rate_score(Respiratory_Rate,config_file)
    score += Oxygenation_score(FiO2,PaO2,Agradient,config_file)
    score += pH_score(pH,config_file)
    score += Sodium_score(Sodium,config_file)
    score += Potassium_score(Potassium,config_file)
    score += Creatinine_score(Creatinine,Renal_Failure,config_file)
    score += Hematocrit_score(Hematocrit,config_file)
    score += White_Blood_score(White_Blood,config_file)
    score += float(Glasgow_Coma_Scale)
    
    
    return score


### Testing your Function

Write a few test cases to make sure that your code functions correctly.  In the last step, you'll have LOTS of test cases run through, but you should do some of your before moving on.

In [462]:
assert(apache_score(["None"],51,7,97,30,28,70,
               460,8,10,3.6,1.4,"Chronic Renal Failure",48,41,14)==35.0)

---

## Part 4: Accessing and processing the patient file

Fill out the simple function below to retrieve the patient data as a CSV file from any given URL and return a list of all of the Apache II scores based on the data you find for those patients.
* The patient file will be a CSV
* It will have column headers that match the labels shown above
* The columns will not necessarily appear in the order shown above
* You should output only the Apache II scores, not any other information
* Your output should be a list in the same order as the input rows

In [442]:
import requests
import json
import csv
with open('Scores_new.csv') as a:
    A=[]
    s = list(csv.reader(a))
    for i in range(1,len(s)):
        A.append(s[i])

In [443]:
Answer=[]
for j in A:
    Answer.append(float(j[0]))
    

In [444]:
import json
import requests
import csv


with open('TestPatients_new.csv') as f:
    r = csv.reader(f)
    #next(r, None)
    r=list(r)
    z=r.pop(0)
    t=r[60]
    scores=[]
    for line in r:
        organ_failure_list = line[1]
        Age = line[2]  
        Temperature = line[3]
        pH = line[4]
        Heart_Rate = line[5]
        Respiratory_Rate = line[6]  
        Sodium = line[7]
        Potassium = line[8]  
        Creatinine= line[9]
        Renal_Failure = line[10]  
        Hematocrit = line[11]
        White_Blood = line[12]  
        Glasgow_Coma_Scale = line[13]
        FiO2 = line[14]  
        PaO2 = line[15]
        Agradient = line[16]  
        score = apache_score([organ_failure_list],Age,Temperature,Heart_Rate,Respiratory_Rate,FiO2,PaO2,
                Agradient,pH,Sodium,Potassium,Creatinine,Renal_Failure,Hematocrit,White_Blood,Glasgow_Coma_Scale)
        scores.append(score)
    
    


In [459]:
assert(scores == Answer)

AssertionError: 

In [438]:
        organ_failure_list = t[1]
        Age = t[2]  
        Temperature = t[3]
        pH = t[4]
        Heart_Rate = t[5]
        Respiratory_Rate = t[6]  
        Sodium = t[7]
        Potassium = t[8]  
        Creatinine= t[9]
        Renal_Failure = t[10]  
        Hematocrit = t[11]
        White_Blood = t[12]  
        Glasgow_Coma_Scale = t[13]
        FiO2 = t[14]  
        PaO2 = t[15]
        Agradient = t[16]  
        score = apache_score([organ_failure_list],Age,Temperature,Heart_Rate,Respiratory_Rate,FiO2,PaO2,
                Agradient,pH,Sodium,Potassium,Creatinine,Renal_Failure,Hematocrit,White_Blood,Glasgow_Coma_Scale)

In [439]:
score

35.0

In [451]:
G=[]
for x in range(0,len(scores)):
    if scores[x] != Answer[x]:
            G.append(x)

In [452]:
G

[82, 251]

In [455]:
scores[82]

38.0

In [456]:
Answer[82]

42.0

In [419]:
import csv
A=scores
score_count=0
with open('scores_edit.csv','w') as csvfile:
    filednames=['Total']
    thewriter=csv.DictWriter(csvfile,filednames)
    thewriter.writeheader()
    for score in A:
        thewriter.writerow({'Total':score})
    
        


### Testing your Function

The URL for the test data is: https://hds5210-2020.s3.amazonaws.com/TestPatients.csv


You can verify your results by comparing them against this data: https://hds5210-2020.s3.amazonaws.com/Scores.csv

**CORRECTION ADDED 3/29** - If you calculated the Glasgow Coma Scale points as per the actual instructions in MDCalc, then please use this set of corrected scores to compare your results with: https://hds5210-2020.s3.amazonaws.com/Scores_corrected.csv
